# Q4

In [12]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 7
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [13]:
main() 


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.300039
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.213470
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.170460
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.076698
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.868079
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.414145
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.000679
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.776571
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.459994
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.486537
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.438622
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.408670
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.459864
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.429242
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.398291
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.384615
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.297624
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.503118
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.523674
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.338097
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.150661
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.102862
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.136994
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.141804
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.053474
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.053494
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.027823
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.071233
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.101155
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.029726
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.039104
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.024843
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.040786
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.188778
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.063856
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.044919
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.021296
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.068275
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.082650
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.038054


Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.058255
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.008491
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.033305
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.014452
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.012224
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.128874
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.059160
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.083852
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.076794
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.028648
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.132420
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.008943
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.031623
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.010481
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.011646
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.053334
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.150252
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.011496
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.034441
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.005794


Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.030894
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.006305
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.011748
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.015598
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.013499
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.012683
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.053479
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.022271
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.017292
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.031661
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.039483
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.001994
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.058042
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.027234
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.003629
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.019212
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.008633
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.022586
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.061499
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.046820


Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.013725
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.035247

Test set: Average loss: 0.0338, Accuracy: 9882/10000 (99%)



After 7th epoch, Average test loss was 0.0338, accuracy was 9882/10000 (99%)

## (A)

Replacing ReLU with sigmoid

In [14]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.sigmoid(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.sigmoid(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.sigmoid(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 7
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [15]:
main()

/home/supreets51/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.342637
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.275414
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.325428
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.342218
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.302986
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.315792
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.299521
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.296384
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.313894
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.295561
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.333241
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.324884
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.331858
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.296161
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.313997
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.307092
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.299965
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.321686
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.336412
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.300013
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 2.290636
Train Epoch: 2 [45440/60000 (76%)]	Loss: 2.303389
Train Epoch: 2 [46080/60000 (77%)]	Loss: 2.277572
Train Epoch: 2 [46720/60000 (78%)]	Loss: 2.321103
Train Epoch: 2 [47360/60000 (79%)]	Loss: 2.289357
Train Epoch: 2 [48000/60000 (80%)]	Loss: 2.292134
Train Epoch: 2 [48640/60000 (81%)]	Loss: 2.296438
Train Epoch: 2 [49280/60000 (82%)]	Loss: 2.321246
Train Epoch: 2 [49920/60000 (83%)]	Loss: 2.267919
Train Epoch: 2 [50560/60000 (84%)]	Loss: 2.279750
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.293864
Train Epoch: 2 [51840/60000 (86%)]	Loss: 2.272770
Train Epoch: 2 [52480/60000 (87%)]	Loss: 2.301153
Train Epoch: 2 [53120/60000 (88%)]	Loss: 2.295271
Train Epoch: 2 [53760/60000 (90%)]	Loss: 2.284942
Train Epoch: 2 [54400/60000 (91%)]	Loss: 2.298219
Train Epoch: 2 [55040/60000 (92%)]	Loss: 2.280043
Train Epoch: 2 [55680/60000 (93%)]	Loss: 2.245434
Train Epoch: 2 [56320/60000 (94%)]	Loss: 2.278687
Train Epoch: 2 [56960/60000 (95%)]	Loss: 2.233397


Train Epoch: 4 [28800/60000 (48%)]	Loss: 1.330223
Train Epoch: 4 [29440/60000 (49%)]	Loss: 1.303635
Train Epoch: 4 [30080/60000 (50%)]	Loss: 1.273924
Train Epoch: 4 [30720/60000 (51%)]	Loss: 1.239628
Train Epoch: 4 [31360/60000 (52%)]	Loss: 1.215446
Train Epoch: 4 [32000/60000 (53%)]	Loss: 1.198616
Train Epoch: 4 [32640/60000 (54%)]	Loss: 1.335434
Train Epoch: 4 [33280/60000 (55%)]	Loss: 1.310926
Train Epoch: 4 [33920/60000 (57%)]	Loss: 1.273864
Train Epoch: 4 [34560/60000 (58%)]	Loss: 1.377065
Train Epoch: 4 [35200/60000 (59%)]	Loss: 1.285352
Train Epoch: 4 [35840/60000 (60%)]	Loss: 1.196230
Train Epoch: 4 [36480/60000 (61%)]	Loss: 1.160928
Train Epoch: 4 [37120/60000 (62%)]	Loss: 1.231034
Train Epoch: 4 [37760/60000 (63%)]	Loss: 1.118833
Train Epoch: 4 [38400/60000 (64%)]	Loss: 1.208870
Train Epoch: 4 [39040/60000 (65%)]	Loss: 1.059954
Train Epoch: 4 [39680/60000 (66%)]	Loss: 1.057809
Train Epoch: 4 [40320/60000 (67%)]	Loss: 1.120212
Train Epoch: 4 [40960/60000 (68%)]	Loss: 1.132311


Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.532925
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.433662
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.484852
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.486917
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.477583
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.517330
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.444764
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.512444
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.344798
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.609266
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.545816
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.439663
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.542868
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.444450
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.705224
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.607984
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.358931
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.428908
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.428142
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.536383


Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.223377
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.398007
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.291399
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.378778

Test set: Average loss: 0.3411, Accuracy: 9007/10000 (90%)



### Observation

Sigmoid functions performed worse, with 90% accuracy (compared to 99% of ReLU). 

This could be because, in sigmoid gradients can vanish faster compared to relu. That might be the case here, due to more number of layers 

## (B)

Added one dropout layer after the convolution layers.

### Dropout = 0.25

In [18]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.drop_out = nn.Dropout(0.25)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.drop_out(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 4
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [19]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302438
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.214872
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.191051
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.146497
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.008911
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.647428
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.233815
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.961808
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.706661
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.623777
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.579547
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.559234
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.605837
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.573964
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.549593
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.502959
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.338784
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.570220
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.565026
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.394789
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.076771
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.196752
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.157548
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.094730
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.070347
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.025220
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.120698
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.143957
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.053797
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.098873
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.024901
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.035850
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.193051
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.079137
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.045590
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.040993
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.077186
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.098562
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.048867
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.208529


Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.080286
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.014770
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.050238
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.028131
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.027925
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.131046
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.066824
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.096203
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.059777
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.050786
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.064303
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.018228
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.087861
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.025660
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.026093
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.028311
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.122733
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.012031
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.081592
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.004265


#### For droupout 0.25

Avg Train Loss: 0.05
Test Accuracy: 98.74%

### Dropout = 0.5

In [20]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.drop_out = nn.Dropout(0.5)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.drop_out(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 4
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [21]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.288615
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.231679
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.215999
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.193267
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.083044
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.813028
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.548236
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.291320
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.984300
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.833900
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.809248
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.728571
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.721903
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.746094
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.624723
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.597086
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.468533
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.570441
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.646953
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.500395
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.059818
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.186656
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.097676
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.156983
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.201597
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.137416
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.092061
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.056309
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.059558
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.211817
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.054249
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.083855
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.055101
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.089853
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.200411
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.105806
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.087297
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.034521
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.094030
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.127229


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.063386
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.010014
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.074632
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.015501
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.137164
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.038397
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.042762
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.171524
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.052190
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.108835
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.091424
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.077826
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.081915
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.071321
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.124340
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.078062
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.034429
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.086792
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.179954
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.040597


#### Observation

Test accuracy decreaseed slightly to 98.60%. Training Loss also increased


### Dropout = 0.75

In [22]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.drop_out = nn.Dropout(0.75)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.drop_out(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 4
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [23]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299838
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.248612
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.243206
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.285064
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.150430
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.983510
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.926721
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.806068
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.553669
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.187700
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.247060
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.057142
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.995211
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.025818
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.861474
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.777349
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.683843
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.667443
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.845052
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.656947
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.146576
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.362393
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.136588
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.279138
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.301215
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.277935
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.159851
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.140154
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.185647
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.286530
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.071218
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.128399
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.150193
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.089101
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.326131
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.131239
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.177768
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.064316
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.097458
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.175407


Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.075604
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.025005
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.050394
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.048206
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.141046
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.080736
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.096836
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.315587
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.144390
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.082262
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.103050
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.145949
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.149240
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.055067
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.089675
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.081284
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.031120
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.159605
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.153603
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.040787


Test accuracy further decreased. 

### Dropout 1

In [24]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        self.drop_out = nn.Dropout(1)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.drop_out(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 4
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [25]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302219
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.298826
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.301970
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.308567
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.304008
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.297559
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.302481
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.307774
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.300972
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.303229
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.304164
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.299875
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.305195
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.299471
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.301483
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.303050
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.308152
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.307173
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.302831
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.301004
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 2.304663
Train Epoch: 2 [45440/60000 (76%)]	Loss: 2.304406
Train Epoch: 2 [46080/60000 (77%)]	Loss: 2.302018
Train Epoch: 2 [46720/60000 (78%)]	Loss: 2.302035
Train Epoch: 2 [47360/60000 (79%)]	Loss: 2.302495
Train Epoch: 2 [48000/60000 (80%)]	Loss: 2.308725
Train Epoch: 2 [48640/60000 (81%)]	Loss: 2.297300
Train Epoch: 2 [49280/60000 (82%)]	Loss: 2.310734
Train Epoch: 2 [49920/60000 (83%)]	Loss: 2.292474
Train Epoch: 2 [50560/60000 (84%)]	Loss: 2.304361
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.304205
Train Epoch: 2 [51840/60000 (86%)]	Loss: 2.303553
Train Epoch: 2 [52480/60000 (87%)]	Loss: 2.297390
Train Epoch: 2 [53120/60000 (88%)]	Loss: 2.300995
Train Epoch: 2 [53760/60000 (90%)]	Loss: 2.303013
Train Epoch: 2 [54400/60000 (91%)]	Loss: 2.305496
Train Epoch: 2 [55040/60000 (92%)]	Loss: 2.304936
Train Epoch: 2 [55680/60000 (93%)]	Loss: 2.296433
Train Epoch: 2 [56320/60000 (94%)]	Loss: 2.307404
Train Epoch: 2 [56960/60000 (95%)]	Loss: 2.293394


Train Epoch: 4 [30080/60000 (50%)]	Loss: 2.305728
Train Epoch: 4 [30720/60000 (51%)]	Loss: 2.301325
Train Epoch: 4 [31360/60000 (52%)]	Loss: 2.304516
Train Epoch: 4 [32000/60000 (53%)]	Loss: 2.296780
Train Epoch: 4 [32640/60000 (54%)]	Loss: 2.295546
Train Epoch: 4 [33280/60000 (55%)]	Loss: 2.296950
Train Epoch: 4 [33920/60000 (57%)]	Loss: 2.301523
Train Epoch: 4 [34560/60000 (58%)]	Loss: 2.308598
Train Epoch: 4 [35200/60000 (59%)]	Loss: 2.308405
Train Epoch: 4 [35840/60000 (60%)]	Loss: 2.302159
Train Epoch: 4 [36480/60000 (61%)]	Loss: 2.298808
Train Epoch: 4 [37120/60000 (62%)]	Loss: 2.311924
Train Epoch: 4 [37760/60000 (63%)]	Loss: 2.310860
Train Epoch: 4 [38400/60000 (64%)]	Loss: 2.301408
Train Epoch: 4 [39040/60000 (65%)]	Loss: 2.300657
Train Epoch: 4 [39680/60000 (66%)]	Loss: 2.298049
Train Epoch: 4 [40320/60000 (67%)]	Loss: 2.292927
Train Epoch: 4 [40960/60000 (68%)]	Loss: 2.305258
Train Epoch: 4 [41600/60000 (69%)]	Loss: 2.297454
Train Epoch: 4 [42240/60000 (70%)]	Loss: 2.299268


This leads to a very poor accuracy, since we are zeroing out elements with probability 1.

### Results

Dropout probability 0.25 performed the best. This should be expected, because very high dropout percentage results in over-regularising : a neuron can't perform its task as well as it should do.

## With Batch Normalization

Two Batch norm layers - after 2nd convolution layer, 1 after 1st fc layer.

Dropout = 0.25

In [30]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc1_bn = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)
        self.drop_out = nn.Dropout(0.25)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = self.drop_out(x)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 4
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [31]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.418815
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.454403
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.177809
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.932977
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.876680
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.711482
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.526262
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.430519
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.305108
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.344176
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.333011
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.373480
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.291408
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.396604
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.315319
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.272555
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.247432
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.346282
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.309101
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.288782
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.097734
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.097561
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.166999
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.133840
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.081329
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.085681
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.024593
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.095358
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.096558
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.063377
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.072314
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.046634
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.035486
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.123731
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.116534
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.042362
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.016682
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.088082
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.131566
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.041170


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.009808
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.085165
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.024266
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.027061
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.015888
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.042418
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.089945
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.041226
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.048129
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.025014
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.036655
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.069791
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.021215
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.039338
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.068995
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.042388
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.036935
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.055708
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.010235
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.035906


Training performance seems to have improved. Test accuracy was 99.06%. 

### Removing dropout

In [35]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_bn = nn.BatchNorm2d(50)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc1_bn = nn.BatchNorm1d(500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 4
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [36]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.409055
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.079869
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.834873
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.557010
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.586559
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.520790
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.339937
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.284065
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.199713
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.199060
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.217041
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.206426
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.202178
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.299025
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.203627
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.173473
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.205702
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.277533
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.213907
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.190402
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.095305
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.106300
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.097276
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.073684
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.058219
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.043925
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.014231
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.041110
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.078981
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.021248
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.056143
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.027169
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.034990
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.053076
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.042101
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.025828
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.011304
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.040947
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.064204
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.037805


Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.006075
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.038769
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.010471
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.013479
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.008554
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.023115
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.045167
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.038785
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.054543
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.025546
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.014335
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.040981
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.006686
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.019695
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.034746
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.009527
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.023441
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.070193
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.006227
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.005525


The accuracy is similar to that without dropout. 

## (D)

Adding xavier_normal initialization to the layers with batch normalization.

In [37]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_bn = nn.BatchNorm2d(50)
        torch.nn.init.xavier_normal(self.conv2.weight)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc1_bn = nn.BatchNorm1d(500)
        torch.nn.init.xavier_normal(self.fc1.weight)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 4
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")

In [38]:
main()

/home/supreets51/.local/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  app.launch_new_instance()
/home/supreets51/.local/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.327536
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.515190
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.050700
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.694778
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.634469
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.596245
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.480003
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.441170
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.379675
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.371906
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.338966
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.355591
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.345982
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.365335
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.274998
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.246717
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.254600
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.302908
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.261223
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.210433
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.058244
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.033071
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.083437
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.079857
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.068835
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.072753
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.065397
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.101655
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.022358
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.139903
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.068494
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.103345
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.074353
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.051588
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.059363
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.072292
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.082124
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.058443
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.122582
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.132746


Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.027844
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.105384
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.015235
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.019039
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.019434
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.024712
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.041213
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.023077
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.037329
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.057746
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.004069
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.005938
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.029021
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.020571
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.032711
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.031048
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.012321
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.081036
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.075923
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.046389


Again the accuracy remains almost unchanged ~99% (compared to previous parts, where no explicit initialization of weights was done).

### Using kaiming_uniform_

In [41]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.conv2_bn = nn.BatchNorm2d(50)
        torch.nn.init.kaiming_normal_(self.conv2.weight)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc1_bn = nn.BatchNorm1d(500)
        torch.nn.init.kaiming_normal_(self.fc1.weight)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():

    use_cuda = not False and torch.cuda.is_available()

    torch.manual_seed(1)
    
    batch_sz = 64
    
    lrate = 0.01
    
    epcs = 4
    
    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_sz, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=1000, shuffle=True, **kwargs)


    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lrate, momentum=0.5)

    for epoch in range(1, epcs + 1):
        train(None, model, device, train_loader, optimizer, epoch)
        test(None, model, device, test_loader)

    if (False):
        torch.save(model.state_dict(),"mnist_cnn.pt")


In [42]:
main()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.327537
Train Epoch: 1 [640/60000 (1%)]	Loss: 1.688275
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.240651
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.855569
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.733452
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.718369
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.587912
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.548990
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.474070
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.453363
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.425023
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.405428
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.425091
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.423196
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.333802
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.315034
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.339806
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.362763
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.331138
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.284595
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.093070
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.089462
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.080098
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.092496
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.076185
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.118254
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.037728
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.203723
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.085927
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.123676
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.134554
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.069681
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.080245
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.091720
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.096756
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.059163
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.145884
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.155617
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.095527
Train Epoch: 2 [58240/60000 (97%)]	Loss: 0.126102


Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.046062
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.112438
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.021905
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.026871
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.017628
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.028762
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.056695
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.034789
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.095988
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.076530
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.007736
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.008560
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.034736
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.042501
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.044195
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.068859
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.021691
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.101005
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.090291
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.049951


## Observations

Sigmoid activation functions don't work very well here due to vanishing gradient problem.

High dropout leads to a drop in performance since we are not allowing it to train fully. However, small dropout ~0.25 leads to a slight increase in accuracy, maybe because it is a regularizer.